In [1]:
import os
from processing import split_train_test
import tensorflow as tf

In [2]:
filepaths = ['data/airplane.ndjson', 'data/apple.ndjson', 'data/umbrella.ndjson']
'''
for filepath in filepaths:
    split_train_test(filepath, 80, 20)
'''

'\nfor filepath in filepaths:\n    split_train_test(filepath, 80, 20)\n'

In [3]:
classnames = ['airplane', 'apple', 'umbrella']

for classname in classnames:
    classdir = './images/train/{}'.format(classname)
    print('total {} images: {}'.format(classname, len(os.listdir(classdir))))

total airplane images: 108663
total apple images: 111922
total umbrella images: 96414


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
batch_size = 64

train_datagen = ImageDataGenerator()
train_flow = train_datagen.flow_from_directory(
    './images/train', # source category directory
    batch_size = batch_size,
    classes = classnames,
    class_mode = 'categorical')

Found 316999 images belonging to 3 classes.


In [6]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [7]:
input_layer = Input(shape=(256, 256, 3))

convolution_layers = [
    {
        'output': 16,
        'k_size': (3, 3),
        'active': 'relu',
        'pool_size': 2,
        'stride': 2,
    },
    {
        'output': 32,
        'k_size': (3, 3),
        'active': 'relu',
        'pool_size': 2,
        'stride': 2,
    },
    {
        'output': 64,
        'k_size': (3, 3),
        'active': 'relu',
        'pool_size': 2,
        'stride': 2,
    },
    {
        'output': 64,
        'k_size': (3, 3),
        'active': 'relu',
        'pool_size': 2,
        'stride': 2,
    },
    {
        'output': 64,
        'k_size': (3, 3),
        'active': 'relu',
        'pool_size': 2,
        'stride': 2,
    },
]

current_layer = input_layer

for layer in convolution_layers:
    current_layer = Conv2D(layer['output'], layer['k_size'], activation=layer['active'])(current_layer)
    current_layer = MaxPooling2D(layer['pool_size'], layer['stride'])(current_layer)

current_layer = Flatten()(current_layer)
current_layer = Dense(128, activation='relu')(current_layer)
output_layer = Dense(len(classnames), activation='softmax')(current_layer)

model = Model(inputs=input_layer, outputs=output_layer)


In [8]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 254, 254, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 64)        0     

In [9]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [ ]:
test_datagen = ImageDataGenerator()
test_flow = test_datagen.flow_from_directory(
    './images/test', # source category directory
    batch_size = batch_size,
    classes = classnames,
    class_mode = 'categorical')

n_epochs = 30
n_samples = train_flow.n
filepath = filepath="models/drawing-classifier-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath, verbose=1)

history = model.fit_generator(
    train_flow,
    steps_per_epoch=n_samples // batch_size,
    epochs=n_epochs,
    verbose=1,
    use_multiprocessing=True,
    validation_data=test_flow,
    callbacks=[checkpoint],
)

Found 79240 images belonging to 3 classes.
Epoch 1/30
2726/4953 [===============>..............] - ETA: 1:30:27 - loss: 11.2356 - accuracy: 0.3029